# <div align="center" style="color: #ff5733;">Sil Account level tagging based on Item Purchased</div>

# Logic for this is to select the data based on the Item purchased. If the Item Purchased in Mobile phone then it is a Mobile store

Created by : Dwaipayan Chakroborti<br>
Requestor: Anastasia -- Refer to her group chat with Heaven, Biswa, Me and Sreekanth<br>
Dated: 2024-09-30

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """     
SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  lmt.new_loan_type,
  lmt.flagDisbursement,
  lmt.disbursementDateTime,
  lmt.disbursedLoanAmount,
  lmt.downPaymentAmount,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
  `prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_FINAL` asf
ON
  UPPER(asf.PURPLE_KEY) = UPPER(lmt.purpleKey) --
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.flagDisbursement = 1
 and lmt.disbursementDateTime is not null
  ;
"""

In [3]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Query is running:   0%|          |

NotFound: 404 Not found: Table prj-prod-dataplatform:worktable_datachampions.AREA_SPLIT_FINAL was not found in location asia-southeast1; reason: notFound, message: Not found: Table prj-prod-dataplatform:worktable_datachampions.AREA_SPLIT_FINAL was not found in location asia-southeast1

Location: asia-southeast1
Job ID: 73454b51-a63b-4acd-9ecc-d286fd5f7b2d


In [6]:
df.to_csv(r"LoanLevelPurchasedITEM.csv", index = False)

# New query with the code part from Marek also included for better clarity

In [4]:
sq = """SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  substr(cast(date_trunc(lmt.decision_date, MONTH) as string),0,10) as mth,
  lmt.employmentstatus,
  case
  when EXTRACT(HOUR FROM lmt.termsAndConditionsSubmitDateTime)<18 then "<18"
  else "18+" end as application_hour,
  case
  when upper(lmt.purposeDescription) like "%MOBILE%" then "mobile"
  else "non-mobile" end as goods_type,
  case
  when c.store_type=1 then "appliance"
  else "mobile" end as pos_type,
  c.mer_name as retailer,
  lmt.new_loan_type,
  lmt.loantype,
  lmt.flagApproval as approved,
  lmt.flagDisbursement as disbursed,
  lmt.flagrejection as rejected,
  lmt.disbursedloanamount as volume_out,
  lmt.disbursedLoanTenur as tenor,
  lmt.disbursementDateTime,
  lmt.downPaymentAmount,
  case
    when lmt.loanRequestAmount<=10000 then "<10k"
    when lmt.loanRequestAmount<=15000 then "<15k"
    when lmt.loanRequestAmount<=20000 then "<20k"
    when lmt.loanRequestAmount<=25000 then "<25k"
    when lmt.loanRequestAmount<=30000 then "<30k"
    else "30k+" end as ca_category,
  case
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.195 then "<20%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.295 then "<30%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.35 then "<35%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.4 then "<40%"
  else "40+" end as DP_range,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.mer_category STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_CURRENTV2 asf
ON
 UPPER(asf.purpleKey) = UPPER(lmt.purpleKey) --
inner join `dl_loans_db_raw.tdbk_merchant_refferal_mtb` c on lmt.purpleKey=c.mer_refferal_code
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.loantype="BNPL"
  ;"""

In [5]:
dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID b44e6cd5-85fd-4892-8dff-c465fd1532f5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [8]:
dfd.shape

(483551, 35)

In [9]:
dfd.columns

Index(['digitalLoanAccountId', 'loanAccountNumber', 'purpleKey',
       'merchantName', 'startApplyDateTime', 'applicationStatus',
       'decision_date', 'mth', 'employmentstatus', 'application_hour',
       'goods_type', 'pos_type', 'retailer', 'new_loan_type', 'loantype',
       'approved', 'disbursed', 'rejected', 'volume_out', 'tenor',
       'disbursementDateTime', 'downPaymentAmount', 'ca_category', 'DP_range',
       'loanCategoryId', 'createdDate', 'brandName', 'skuNo', 'price',
       'product_id', 'product_category', 'product_description', 'STORE_TAG',
       'product_downpayment', 'product_crifcode'],
      dtype='object')

In [10]:
pd.set_option("Display.max_columns", None)
dfd.head()

,digitalLoanAccountId,loanAccountNumber,purpleKey,merchantName,startApplyDateTime,applicationStatus,decision_date,mth,employmentstatus,application_hour,goods_type,pos_type,retailer,new_loan_type,loantype,approved,disbursed,rejected,volume_out,tenor,disbursementDateTime,downPaymentAmount,ca_category,DP_range,loanCategoryId,createdDate,brandName,skuNo,price,product_id,product_category,product_description,STORE_TAG,product_downpayment,product_crifcode
0,200f7dda-2e25-4422-86e9-559ebe8f7032,None,M7TGLXS,Rulls Inline Emall,2024-08-17 18:51:05,RESET,NaT,None,SALARIED,18+,mobile,mobile,Rulls Inline Emall,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,8097.00,<20k,<35%,1747,2024-08-17 10:51:01+00:00,iphone,iphone 11 128gb,24990.00,1747,None,Mobile Phones,Mobile Retailer,20.0,107
1,200f7dda-2e25-4422-86e9-559ebe8f7032,None,M7TGLXS,Rulls Inline Emall,2024-08-17 18:51:05,RESET,NaT,None,SALARIED,18+,mobile,mobile,Rulls Inline Emall,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,8097.00,<20k,<35%,1731,2024-08-17 10:51:01+00:00,mpg,mpg,2000.00,1731,None,Accessories or Peripherals,Mobile Retailer,0.0,126
2,886480d2-424e-48cb-b545-112cf6bd7f33,None,ME3DTAP,Robinsons Appliances Lemery,2024-12-29 15:20:18,RESET,NaT,None,UNEMPLOYED,18+,non-mobile,mobile,Robinsons Appliances Lemery,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,8000.00,<10k,40+,1723,2024-12-29 07:19:59+00:00,american home,aham awf-minv09523g wm Tl,15499.00,1723,None,"Washing machine, dryer or both",Appliances Retailer,0.0,120
3,16d6f754-b165-47f3-bda5-bfc3ce2d9df0,None,MHDQCN6,FC Home Center Zapote,2024-04-07 14:33:37,RESET,NaT,None,SELF EMPLOYED,18+,non-mobile,appliance,FC Home Center Zapote,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,3500.00,<10k,<40%,1719,2024-04-07 06:33:26+00:00,SAMWON,SW-M32PRO,9799.00,1719,None,Televisions,Appliances Retailer,0.0,116
4,3846beda-5213-41b6-a168-9b613e118b70,None,MTPR3CS,Lovely and Julie Home Lucena,2024-11-16 10:44:36,RESET,NaT,None,SELF EMPLOYED,18+,non-mobile,appliance,Lovely and Julie Home Lucena,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,1000.00,<10k,<20%,1723,2024-11-16 02:44:28+00:00,xtreme,xwmst-0008,3800.00,1723,None,"Washing machine, dryer or both",Appliances Retailer,0.0,120


In [11]:
print(dfd.dtypes)

digitalLoanAccountId                 object
loanAccountNumber                    object
purpleKey                            object
merchantName                         object
startApplyDateTime           datetime64[us]
applicationStatus                    object
decision_date                datetime64[us]
mth                                  object
employmentstatus                     object
application_hour                     object
goods_type                           object
pos_type                             object
retailer                             object
new_loan_type                        object
loantype                             object
approved                              Int64
disbursed                             Int64
rejected                              Int64
volume_out                          float64
tenor                                 Int64
disbursementDateTime         datetime64[us]
downPaymentAmount                    object
ca_category                     

In [12]:
datetime_columns = ['startApplyDateTime', 'decision_date', 'disbursementDateTime', 'createdDate']

# Remove timezone information
for col in datetime_columns:
    dfd[col] = dfd[col].dt.tz_localize(None)
    


In [13]:
dfd['new_loan_type'].value_counts()

new_loan_type
SIL-Instore                 455099
SIL ZERO                     16948
SIL Competitor                8693
SIL Repeat                    2809
SIL-Online(discontinued)         2
Name: count, dtype: int64

In [14]:
dfd[dfd['new_loan_type']=='SIL-Online(discontinued)']

,digitalLoanAccountId,loanAccountNumber,purpleKey,merchantName,startApplyDateTime,applicationStatus,decision_date,mth,employmentstatus,application_hour,goods_type,pos_type,retailer,new_loan_type,loantype,approved,disbursed,rejected,volume_out,tenor,disbursementDateTime,downPaymentAmount,ca_category,DP_range,loanCategoryId,createdDate,brandName,skuNo,price,product_id,product_category,product_description,STORE_TAG,product_downpayment,product_crifcode
36214,f7e37afb-2bb5-4c74-b6c5-cbebcc1fe149,60816359300013,MUSHNEX,Home Along Cainta,2022-09-22 13:39:36,COMPLETED,2022-09-22 15:26:06,2022-09-01,SALARIED,<18,non-mobile,appliance,Home Along Cainta,SIL-Online(discontinued),BNPL,1,1,0,16000.0,24,2022-09-22 15:53:55,None,<20k,40+,1721,2022-10-27 16:50:24,None,None,None,1721,None,Refrigerators,Home Appliances Retailer,0.0,119
87943,3ca1c00f-bbcd-458d-ab43-6129778a1f8b,None,MX0OJ4A,Home Along Malabon,NaT,INITIATE,NaT,None,SALARIED,18+,non-mobile,appliance,Home Along Malabon,SIL-Online(discontinued),BNPL,0,0,0,NaN,<NA>,NaT,None,<25k,40+,1719,2022-10-27 16:50:24,None,None,None,1719,None,Televisions,Home Appliances Retailer,0.0,116


In [15]:
dfd.to_excel("Sil_DisbursedData_20250204.xlsx", index = False)

In [16]:
dfd[dfd['disbursementDateTime'].isnull()]

,digitalLoanAccountId,loanAccountNumber,purpleKey,merchantName,startApplyDateTime,applicationStatus,decision_date,mth,employmentstatus,application_hour,goods_type,pos_type,retailer,new_loan_type,loantype,approved,disbursed,rejected,volume_out,tenor,disbursementDateTime,downPaymentAmount,ca_category,DP_range,loanCategoryId,createdDate,brandName,skuNo,price,product_id,product_category,product_description,STORE_TAG,product_downpayment,product_crifcode
0,200f7dda-2e25-4422-86e9-559ebe8f7032,None,M7TGLXS,Rulls Inline Emall,2024-08-17 18:51:05,RESET,NaT,None,SALARIED,18+,mobile,mobile,Rulls Inline Emall,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,8097.00,<20k,<35%,1747,2024-08-17 10:51:01,iphone,iphone 11 128gb,24990.00,1747,None,Mobile Phones,Mobile Retailer,20.0,107
1,200f7dda-2e25-4422-86e9-559ebe8f7032,None,M7TGLXS,Rulls Inline Emall,2024-08-17 18:51:05,RESET,NaT,None,SALARIED,18+,mobile,mobile,Rulls Inline Emall,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,8097.00,<20k,<35%,1731,2024-08-17 10:51:01,mpg,mpg,2000.00,1731,None,Accessories or Peripherals,Mobile Retailer,0.0,126
2,886480d2-424e-48cb-b545-112cf6bd7f33,None,ME3DTAP,Robinsons Appliances Lemery,2024-12-29 15:20:18,RESET,NaT,None,UNEMPLOYED,18+,non-mobile,mobile,Robinsons Appliances Lemery,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,8000.00,<10k,40+,1723,2024-12-29 07:19:59,american home,aham awf-minv09523g wm Tl,15499.00,1723,None,"Washing machine, dryer or both",Appliances Retailer,0.0,120
3,16d6f754-b165-47f3-bda5-bfc3ce2d9df0,None,MHDQCN6,FC Home Center Zapote,2024-04-07 14:33:37,RESET,NaT,None,SELF EMPLOYED,18+,non-mobile,appliance,FC Home Center Zapote,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,3500.00,<10k,<40%,1719,2024-04-07 06:33:26,SAMWON,SW-M32PRO,9799.00,1719,None,Televisions,Appliances Retailer,0.0,116
4,3846beda-5213-41b6-a168-9b613e118b70,None,MTPR3CS,Lovely and Julie Home Lucena,2024-11-16 10:44:36,RESET,NaT,None,SELF EMPLOYED,18+,non-mobile,appliance,Lovely and Julie Home Lucena,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,1000.00,<10k,<20%,1723,2024-11-16 02:44:28,xtreme,xwmst-0008,3800.00,1723,None,"Washing machine, dryer or both",Appliances Retailer,0.0,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483546,3684250e-510d-4e35-91f2-fd4ab09a80ef,None,MYRK3EJ,FC Home Center Binan,2023-12-17 18:25:14,REJECT,2023-12-17 18:28:20,2023-12-01,SALARIED,18+,non-mobile,appliance,FC Home Center Binan,SIL-Instore,BNPL,0,0,1,NaN,<NA>,NaT,1300.00,<10k,<30%,1728,2023-12-17 10:25:00,Prime Foam,7*54*75,5919.00,1728,None,Bed frames or Cribs,Appliances Retailer,0.0,121
483547,3f81764c-1ab9-408a-a8bd-d82bc0dd535e,None,MI9KODE,FC Home Center Guiguinto,2024-10-13 14:19:49,REJECT,2024-10-13 14:23:57,2024-10-01,SALARIED,<18,mobile,appliance,FC Home Center Guiguinto,SIL-Instore,BNPL,0,0,1,NaN,<NA>,NaT,2400.00,<10k,<35%,1747,2024-10-13 06:19:45,Vivo smart phone,Y28 8gb+128gb,7999.00,1747,None,Mobile Phones,Appliances Retailer,20.0,107
483548,0476ca21-e5ca-4379-9428-b0e7f19a623c,None,M29GPT8,FC Home Center Novaliches 1,2023-11-27 15:40:45,REJECT,2023-11-27 15:47:25,2023-11-01,SALARIED,<18,non-mobile,appliance,FC Home Center Novaliches 1,SIL-Instore,BNPL,0,0,1,NaN,<NA>,NaT,2000.00,<10k,<30%,1719,2023-11-27 07:40:36,PRESTIZ,32IHA04SVM,9999.00,1719,None,Televisions,Appliances Retailer,0.0,116
483549,55084a81-42ad-4814-a8e9-3ca055b02e96,None,MFRL8YN,FC Home Center Alapan,2024-06-29 18:43:00,REJECT,2024-06-29 18:50:20,2024-06-01,SELF EMPLOYED,18+,non-mobile,appliance,FC Home Center Alapan,SIL-Instore,BNPL,0,0,1,NaN,<NA>,NaT,1700.00,<10k,<30%,1720,2024-06-29 10:42:19,DB AUDIO,Db audio aurabeats210 amplified speaker,8459.00,1720,None,"Stereo set, Soundbar, Audio equipment",Appliances Retailer,0.0,114
